In [11]:
from monostable_model import ion_channel_model, model_force_square, random_force_gauss
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, fixed, SelectionSlider


interact(
    ion_channel_model,
    a=FloatSlider(min=0.1, max=10.0, step=0.1, value=1, description='a'),
    closed=fixed((-1, 5000)),  # Fixed for now; could be customized further
    opened=fixed((1, 2500)),  # Fixed for now; could be customized further
    D=FloatSlider(min=0.01, max=3.0, step=0.01, value=0.5, description='D'),
    delta_t=SelectionSlider(
        options=[10**-i for i in range(1, 4)],
        value=0.01,
        description='Delta t',
        format="0.0e"
    ),    records=IntSlider(min=1000, max=100000, step=1000, value=50000, description='Records'),
    model_force=Dropdown(
        options={'Square Force': model_force_square, 'Other Force': lambda x: x**2},
        value=model_force_square,
        description='Model Force',
    ),
    random_force=fixed(random_force_gauss),  # Keep it fixed for simplicity
)

interactive(children=(FloatSlider(value=1.0, description='a', max=10.0, min=0.1), FloatSlider(value=0.5, descr…

<function monostable_model.ion_channel_model(a=1, closed=(-1, 5000), opened=(1, 2500), D=0.5, delta_t=0.01, records=50000, model_force=<function model_force_square at 0x7fa794b9c680>, random_force=<function random_force_gauss at 0x7fa79c09cfe0>)>